<a href="https://colab.research.google.com/github/ergysmedaunipd/thesis/blob/main/ThesisUnipdSNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tonic
!pip install snntorch
!pip install psutil

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 4.7 MB/s eta 0:00:00


In [ ]:
def prepare_nmnist_data(inputs, labels, device, n_timesteps=100):
    """
    Modified data preparation with consistent device placement.
    """
    batch_size = inputs.shape[0]

    # Move inputs to device first
    inputs = inputs.to(device)
    labels = labels.to(device)  # Move labels to device

    # Reshape inputs to [time, features, batch]
    inputs = inputs.float().reshape(batch_size, -1, sensor_size[0] * sensor_size[1])
    inputs = inputs.permute(1, 2, 0).contiguous()

    # Handle timesteps
    if inputs.shape[0] > n_timesteps:
        inputs = inputs[:n_timesteps]
    elif inputs.shape[0] < n_timesteps:
        padding = torch.zeros((n_timesteps - inputs.shape[0], inputs.shape[1], batch_size),
                            device=device)
        inputs = torch.cat((inputs, padding), dim=0)

    # One-hot encode labels (now labels are already on correct device)
    labels_onehot = torch.zeros((10, batch_size), device=device)
    labels_onehot.scatter_(0, labels.unsqueeze(0), 1)

    return inputs, labels_onehot

def train(model, trainloader, num_epochs):

    # Warming phase
    print("\nWarming Phase:")
    warming_losses = []
    for epoch in range(5):
        print(f'Warming Epoch [{epoch+1}/2]')
        epoch_losses = []
        epoch_accuracies = []

        for batch_idx, (inputs, labels) in enumerate(trainloader):

            # Prepare data
            inputs, labels = prepare_nmnist_data(inputs, labels, device)

            # Warming step
            loss, predictions = model.warming(inputs, labels)

            # Ensure predictions and labels have correct shape
            # predictions: [batch_size, num_classes]
            # labels: [num_classes, batch_size] -> [batch_size, num_classes]
            labels = labels.t()

            # Calculate accuracy
            pred_classes = torch.argmax(predictions, dim=1)  # [batch_size]
            true_classes = torch.argmax(labels, dim=1)      # [batch_size]
            accuracy = (pred_classes == true_classes).float().mean().item()

            epoch_losses.append(loss)
            epoch_accuracies.append(accuracy)



        # Epoch summary
        avg_loss = sum(epoch_losses) / len(epoch_losses)
        avg_acc = sum(epoch_accuracies) / len(epoch_accuracies)
        warming_losses.append(avg_loss)
        print(f'  Epoch Summary - Loss: {avg_loss:.6f}, Accuracy: {avg_acc:.4f}')

    # Main training
    print("\nMain Training Phase:")
    training_metrics = []
    for epoch in range(num_epochs):
        print(f'Epoch [{epoch+1}/{num_epochs}]')
        epoch_losses = []
        epoch_accuracies = []

        for batch_idx, (inputs, labels) in enumerate(trainloader):
            print(f'  Batch [{batch_idx+1}/{len(trainloader)}]')

            # Prepare data
            inputs, labels = prepare_nmnist_data(inputs, labels, device)

            # Training step
            loss, predictions = model.fit(inputs, labels)

            # Ensure predictions and labels have correct shape
            labels = labels.t()

            # Calculate accuracy
            pred_classes = torch.argmax(predictions, dim=1)
            true_classes = torch.argmax(labels, dim=1)
            accuracy = (pred_classes == true_classes).float().mean().item()

            epoch_losses.append(loss)
            epoch_accuracies.append(accuracy)



        # Epoch summary
        avg_loss = sum(epoch_losses) / len(epoch_losses)
        avg_acc = sum(epoch_accuracies) / len(epoch_accuracies)
        training_metrics.append({
            'epoch': epoch + 1,
            'loss': avg_loss,
            'accuracy': avg_acc
        })
        print(f'  Epoch Summary - Loss: {avg_loss:.6f}, Accuracy: {avg_acc:.4f}')

    return warming_losses, training_metrics

def evaluate(model, testloader):
    print("\nEvaluation Phase:")
    total_loss = 0
    total_acc = 0
    num_batches = 0
    batch_metrics = []

    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(testloader):
            print(f'  Batch [{batch_idx+1}/{len(testloader)}]')

            # Prepare data
            inputs, labels = prepare_nmnist_data(inputs, labels, device)

            # Forward pass
            loss, predictions = model.evaluate(inputs, labels)

            # Calculate accuracy
            pred_classes = torch.argmax(predictions, dim=0)
            true_classes = torch.argmax(labels, dim=0)
            accuracy = (pred_classes == true_classes).float().mean().item()

            # Store batch metrics
            batch_metrics.append({
                'batch': batch_idx + 1,
                'loss': loss,
                'accuracy': accuracy
            })

            total_loss += loss
            total_acc += accuracy
            num_batches += 1

            print(f'    Loss: {loss:.6f}, Accuracy: {accuracy:.4f}')

    avg_loss = total_loss / num_batches
    avg_acc = total_acc / num_batches

    print(f'\nFinal Evaluation Results:')
    print(f'  Average Loss: {avg_loss:.6f}')
    print(f'  Average Accuracy: {avg_acc:.4f}')

    return avg_loss, avg_acc, batch_metrics

# Initialize model and training
n_timesteps = 100
input_dim = sensor_size[0] * sensor_size[1]
hidden_dims = [128, 10]
n_outputs = 10
# DataLoaders


# Calculate size of 20% of data
train_size = int(0.5 * len(trainset)) // 256 * 256
test_size = int(0.5 * len(testset)) // 256 * 256

from torch.utils.data import Subset
train_subset = Subset(trainset, torch.randperm(len(trainset))[:train_size])
test_subset = Subset(testset, torch.randperm(len(testset))[:test_size])


# Create DataLoaders with the subsets
batch_size = 256
trainloader = DataLoader(train_subset,
                        batch_size=batch_size,
                        collate_fn=tonic.collation.PadTensors(),
                        shuffle=True)
testloader = DataLoader(test_subset,
                       batch_size=batch_size,
                       collate_fn=tonic.collation.PadTensors())

print(f"Original training set size: {len(trainset)}")
print(f"training set size: {len(train_subset)}")
print(f"Original test set size: {len(testset)}")
print(f"test set size: {len(test_subset)}")

# Hyperparameters
rho = 0.05
delta = torch.tensor(0.7, device=device)
theta = torch.tensor(0.15, device=device)

# Create model
print("\nInitializing model...")
model = ADMM_SNN(
    n_samples=batch_size,
    n_timesteps=n_timesteps,
    input_dim=input_dim,
    hidden_dims=hidden_dims,  # Changed final dimension to 10
    n_outputs=10,
    rho=rho,
    delta=delta,
    theta=theta
)
print(model)

# Train model
print("\nStarting training process...")
num_epochs = 20
warming_losses, training_metrics = train(model, trainloader, num_epochs)

# Evaluate model
print("\nEvaluating model...")
test_loss, test_acc, test_metrics = evaluate(model, testloader)

# Print final results
print("\nTraining Summary:")
print(f"  Warming phase final loss: {warming_losses[-1]:.6f}")
print(f"  Training final loss: {training_metrics[-1]['loss']:.6f}")
print(f"  Training final accuracy: {training_metrics[-1]['accuracy']:.4f}")
print("\nTest Results:")
print(f"  Test Loss: {test_loss:.6f}")
print(f"  Test Accuracy: {test_acc:.4f}")

In [ ]:
import torch
import torch.nn as nn
import torchvision
import tonic
import tonic.transforms as transforms
from torch.utils.data import DataLoader
from tonic import DiskCachedDataset
import snntorch as snn
from snntorch import surrogate
from snntorch import functional as SF
from snntorch import utils


# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations
sensor_size = tonic.datasets.NMNIST.sensor_size
frame_transform = transforms.Compose([
    transforms.Denoise(filter_time=10000),
    transforms.ToFrame(sensor_size=sensor_size, time_window=1000)
])

# Load datasets
trainset = tonic.datasets.NMNIST(save_to='./data', transform=frame_transform, train=True)
testset = tonic.datasets.NMNIST(save_to='./data', transform=frame_transform, train=False)





Shapes in __init__:
W[0] shape: torch.Size([100, 1156])
W[1] shape: torch.Size([50, 100])
z[0] shape: torch.Size([300, 128, 100])
z[1] shape: torch.Size([300, 128, 50])
a[0] shape: torch.Size([300, 128, 100])
a[1] shape: torch.Size([300, 128, 50])
Sample data shape: torch.Size([128, 309, 2, 34, 34])
Sample target shape: torch.Size([128])


In [ ]:
def prepare_nmnist_data(inputs, labels, device, n_timesteps=100):
    """
    Modified data preparation with consistent device placement.
    """
    batch_size = inputs.shape[0]

    # Move inputs to device first
    inputs = inputs.to(device)
    labels = labels.to(device)  # Move labels to device

    # Reshape inputs to [time, features, batch]
    inputs = inputs.float().reshape(batch_size, -1, sensor_size[0] * sensor_size[1])
    inputs = inputs.permute(1, 2, 0).contiguous()

    # Handle timesteps
    if inputs.shape[0] > n_timesteps:
        inputs = inputs[:n_timesteps]
    elif inputs.shape[0] < n_timesteps:
        padding = torch.zeros((n_timesteps - inputs.shape[0], inputs.shape[1], batch_size),
                            device=device)
        inputs = torch.cat((inputs, padding), dim=0)

    # One-hot encode labels (now labels are already on correct device)
    labels_onehot = torch.zeros((10, batch_size), device=device)
    labels_onehot.scatter_(0, labels.unsqueeze(0), 1)

    return inputs, labels_onehot

def train(model, trainloader, num_epochs):

    # Warming phase
    print("\nWarming Phase:")
    warming_losses = []
    for epoch in range(5):
        print(f'Warming Epoch [{epoch+1}/2]')
        epoch_losses = []
        epoch_accuracies = []

        for batch_idx, (inputs, labels) in enumerate(trainloader):

            # Prepare data
            inputs, labels = prepare_nmnist_data(inputs, labels, device)

            # Warming step
            loss, predictions = model.warming(inputs, labels)

            # Ensure predictions and labels have correct shape
            # predictions: [batch_size, num_classes]
            # labels: [num_classes, batch_size] -> [batch_size, num_classes]
            labels = labels.t()

            # Calculate accuracy
            pred_classes = torch.argmax(predictions, dim=1)  # [batch_size]
            true_classes = torch.argmax(labels, dim=1)      # [batch_size]
            accuracy = (pred_classes == true_classes).float().mean().item()

            epoch_losses.append(loss)
            epoch_accuracies.append(accuracy)



        # Epoch summary
        avg_loss = sum(epoch_losses) / len(epoch_losses)
        avg_acc = sum(epoch_accuracies) / len(epoch_accuracies)
        warming_losses.append(avg_loss)
        print(f'  Epoch Summary - Loss: {avg_loss:.6f}, Accuracy: {avg_acc:.4f}')

    # Main training
    print("\nMain Training Phase:")
    training_metrics = []
    for epoch in range(num_epochs):
        print(f'Epoch [{epoch+1}/{num_epochs}]')
        epoch_losses = []
        epoch_accuracies = []

        for batch_idx, (inputs, labels) in enumerate(trainloader):
            print(f'  Batch [{batch_idx+1}/{len(trainloader)}]')

            # Prepare data
            inputs, labels = prepare_nmnist_data(inputs, labels, device)

            # Training step
            loss, predictions = model.fit(inputs, labels)

            # Ensure predictions and labels have correct shape
            labels = labels.t()

            # Calculate accuracy
            pred_classes = torch.argmax(predictions, dim=1)
            true_classes = torch.argmax(labels, dim=1)
            accuracy = (pred_classes == true_classes).float().mean().item()

            epoch_losses.append(loss)
            epoch_accuracies.append(accuracy)



        # Epoch summary
        avg_loss = sum(epoch_losses) / len(epoch_losses)
        avg_acc = sum(epoch_accuracies) / len(epoch_accuracies)
        training_metrics.append({
            'epoch': epoch + 1,
            'loss': avg_loss,
            'accuracy': avg_acc
        })
        print(f'  Epoch Summary - Loss: {avg_loss:.6f}, Accuracy: {avg_acc:.4f}')

    return warming_losses, training_metrics

def evaluate(model, testloader):
    print("\nEvaluation Phase:")
    total_loss = 0
    total_acc = 0
    num_batches = 0
    batch_metrics = []

    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(testloader):
            print(f'  Batch [{batch_idx+1}/{len(testloader)}]')

            # Prepare data
            inputs, labels = prepare_nmnist_data(inputs, labels, device)

            # Forward pass
            loss, predictions = model.evaluate(inputs, labels)

            # Calculate accuracy
            pred_classes = torch.argmax(predictions, dim=0)
            true_classes = torch.argmax(labels, dim=0)
            accuracy = (pred_classes == true_classes).float().mean().item()

            # Store batch metrics
            batch_metrics.append({
                'batch': batch_idx + 1,
                'loss': loss,
                'accuracy': accuracy
            })

            total_loss += loss
            total_acc += accuracy
            num_batches += 1

            print(f'    Loss: {loss:.6f}, Accuracy: {accuracy:.4f}')

    avg_loss = total_loss / num_batches
    avg_acc = total_acc / num_batches

    print(f'\nFinal Evaluation Results:')
    print(f'  Average Loss: {avg_loss:.6f}')
    print(f'  Average Accuracy: {avg_acc:.4f}')

    return avg_loss, avg_acc, batch_metrics

# Initialize model and training
n_timesteps = 100
input_dim = sensor_size[0] * sensor_size[1]
hidden_dims = [128, 10]
n_outputs = 10
# DataLoaders


# Calculate size of 20% of data
train_size = int(0.5 * len(trainset)) // 256 * 256
test_size = int(0.5 * len(testset)) // 256 * 256

from torch.utils.data import Subset
train_subset = Subset(trainset, torch.randperm(len(trainset))[:train_size])
test_subset = Subset(testset, torch.randperm(len(testset))[:test_size])


# Create DataLoaders with the subsets
batch_size = 256
trainloader = DataLoader(train_subset,
                        batch_size=batch_size,
                        collate_fn=tonic.collation.PadTensors(),
                        shuffle=True)
testloader = DataLoader(test_subset,
                       batch_size=batch_size,
                       collate_fn=tonic.collation.PadTensors())

print(f"Original training set size: {len(trainset)}")
print(f"training set size: {len(train_subset)}")
print(f"Original test set size: {len(testset)}")
print(f"test set size: {len(test_subset)}")

# Hyperparameters
rho = 0.05
delta = torch.tensor(0.7, device=device)
theta = torch.tensor(0.15, device=device)

# Create model
print("\nInitializing model...")
model = ADMM_SNN(
    n_samples=batch_size,
    n_timesteps=n_timesteps,
    input_dim=input_dim,
    hidden_dims=hidden_dims,  # Changed final dimension to 10
    n_outputs=10,
    rho=rho,
    delta=delta,
    theta=theta
)
print(model)

# Train model
print("\nStarting training process...")
num_epochs = 20
warming_losses, training_metrics = train(model, trainloader, num_epochs)

# Evaluate model
print("\nEvaluating model...")
test_loss, test_acc, test_metrics = evaluate(model, testloader)

# Print final results
print("\nTraining Summary:")
print(f"  Warming phase final loss: {warming_losses[-1]:.6f}")
print(f"  Training final loss: {training_metrics[-1]['loss']:.6f}")
print(f"  Training final accuracy: {training_metrics[-1]['accuracy']:.4f}")
print("\nTest Results:")
print(f"  Test Loss: {test_loss:.6f}")
print(f"  Test Accuracy: {test_acc:.4f}")

Shapes in __init__:
W[0] shape: torch.Size([128, 1156])
W[1] shape: torch.Size([64, 128])
z[0] shape: torch.Size([300, 128, 128])
z[1] shape: torch.Size([300, 128, 64])
a[0] shape: torch.Size([300, 128, 128])
a[1] shape: torch.Size([300, 128, 64])
Sample data shape: torch.Size([128, 310, 2, 34, 34])
Sample target shape: torch.Size([128])
------ Warming Epoch: 0 ------


RuntimeError: mat1 and mat2 shapes cannot be multiplied (128x64 and 128x128)